In [ ]:
"""TRENDYOL - ÜRÜN İSİMLERİ"""
# Kütüphanelerin import edilmesi
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re
import pyodbc


# Chrome tarayıcısı başlatılıyor
driver = webdriver.Chrome()
url = "https://www.trendyol.com/butik/liste/1/kadin"
driver.get(url)
time.sleep(5)

# İşlem yapılacak ürün kategorileri
urunler = ["Süpürge", "Buharlı Ütü", "Robot Süpürge", "Kahve Makinesi", "Mutfak Robotu", "Akıllı Saat",
           "Cep Telefonu", "Powerbank", "Kulaklıklar", "Fotoğraf Makinesi", "Buzdolabı", "Çamaşır Makinesi",
           "Bulaşık Makinesi", "Klima", "Bilgisayarlar", "Tablet", "Monitör", "Klavye", "Mouse", "Saç Düzleştirici",
           "Saç Maşası", "Saç Kurutma Makinesi"]

# MSSQL veritabanına bağlantı oluştur
drivername='ODBC Driver 17 for SQL Server'
server = 'Gamze_Aslan\SQLEXPRESS'
database = 'VTYS'

connection_string =f"""
DRIVER = {drivername};
SERVER = {server};
DATABASE={database};"""

connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

# Her bir ürün kategorisi için işlem yap
for urun in urunler:
    kategori = urun

    # Search bar'a ürün ismi yazılıyor
    search_bar = driver.find_element(By.XPATH, "//*[@id='sfx-discovery-search-suggestions']/div/div[1]/input")
    search_bar.clear()
    search_bar.send_keys(urun)
    search_bar.send_keys(Keys.ENTER)
    time.sleep(5)

    # Toplam ürün adeti bulunuyor
    toplam_urun_adeti_str = driver.find_element(By.XPATH, "//*[@id='search-app']/div/div[1]/div[2]/div[1]/div[1]/div").text
    toplam_urun_adeti = int(re.findall(r'\b[0-9]+\b', toplam_urun_adeti_str)[0])
    if (toplam_urun_adeti == 100 or toplam_urun_adeti > 10000):toplam_urun_adeti = 10000
    print(toplam_urun_adeti)

    # Ürün bilgileri çekiliyor ve MSSQL veritabanına ekleniyor
    for i in range(0, toplam_urun_adeti):
        urun_bilgileri = driver.find_elements(By.CSS_SELECTOR, 'h3.prdct-desc-cntnr-ttl-w.two-line-text')
        urun_marka = urun_bilgileri[i].find_elements(By.CSS_SELECTOR, "span")[0].text
        urun_isim = urun_bilgileri[i].find_elements(By.CSS_SELECTOR, "span")[1].text
        #ürün resim url çek
        print(urun_marka,urun_isim)

        # Bilgileri MSSQL veritabanına ekleyin
        """cursor.execute(f"INSERT INTO UrunIsimleri(Kategori, Marka, Urun_Ismi) VALUES ('{kategori}', '{urun_marka}', '{urun_isim}')")"""

        # Her ürün okunduktan sonra sayfayı aşağı kaydır
        if (i % 3 == 0):
            action = webdriver.ActionChains(driver)
            action.key_down(Keys.SPACE).key_up(Keys.SPACE).perform()
            time.sleep(2)

# Bağlantıları kapat
connection.commit()
connection.close()
driver.quit()
